In [1]:
!pip install transformers

     |████████████████████████████████| 1.8MB 5.8MB/s 
     |████████████████████████████████| 2.9MB 56.8MB/s 
     |████████████████████████████████| 890kB 56.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=bc9d764035e110ed5d872de3551e618db423ef14beb37eaa81b53208819829c1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import os 
os.chdir('/content/drive/My Drive/BERT/NER')

In [3]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [4]:
MAX_LEN = 78

device = torch.device("cuda")

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [6]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):
        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label]*n_subwords)
    
    return tokenized_sentence, labels

In [7]:
tag_values = ['I-tim', 'I-eve', 'B-geo', 'B-org', 'B-tim', 'B-art', 'I-art', 'I-nat', 'B-gpe', 'I-geo', 'O', 'I-per', 'I-gpe', 'B-eve', 'I-org', 'B-per', 'B-nat', 'PAD']
tag2idx = {t:i for i, t in enumerate(tag_values)}

In [8]:
from transformers import BertTokenizer, BertForTokenClassification

model = BertForTokenClassification.from_pretrained("bert-base-cased",
                                                   num_labels=len(tag2idx),
                                                   output_attentions=False,
                                                   output_hidden_states=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [9]:
model.load_state_dict(torch.load("/content/drive/MyDrive/BERT/NER/saved_model/epoch2.pth"))

<All keys matched successfully>

In [10]:
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [11]:
!pip install gradio

     |████████████████████████████████| 1.1MB 4.2MB/s 
     |████████████████████████████████| 215kB 50.6MB/s 
     |████████████████████████████████| 962kB 50.1MB/s 
     |████████████████████████████████| 2.6MB 68.1MB/s 
     |████████████████████████████████| 71kB 12.4MB/s 
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-cp36-none-any.whl size=3374 sha256=aecdff237c7b5d540ee863308eb3d92f7dd417ad088df9630c41e7fbddd1171d
  Stored in directory: /root/.cache/pip/wheels/9f/fc/a7/ab5712c3ace9a8f97276465cc2937316ab8063c1fea488ea77
Successfully built flask-cachebuster


In [ ]:
import gradio as gr

def predict(sentence):
    tokenized_sentence = tokenizer.encode(sentence)
    input_ids = torch.tensor([tokenized_sentence]).cuda()

    with torch.no_grad():
        output = model(input_ids)
    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)[0]

    tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])

    new_tokens, new_tags = [], []

    for token, label in zip(tokens, label_indices):
        if token.startswith('##'):
            new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
            new_tokens.append(token)
            new_tags.append(tag_values[label])

    output = ""
    for token, tag in zip(new_tokens, new_tags):
        output += "{}\t\t{}\n".format(token, tag)
    return output

iface = gr.Interface(fn=predict, inputs="text", outputs="text")
iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://31635.gradio.app
Interface loading below...
